# Plotting Data


Data in PyMoDAQ are featured with a lot of metadata, allowing their proper
description and enabling seamlessly saving/loading to hdf5 files. But what about representation?
Analysis? Exploration?

With Python you usually do this by writing a script and manipulate and plot your data using your
favorite backend (matplotlib, plotly, qt, tkinter, ...) However because PyMoDAQ is highly graphical
you won't need that. PyMoDAQ is featured with various data viewers allowing you to plot any
kind of data. You'll see below some nice examples of how to plot your PyMoDAQ's data using the builtin
data viewers.

## Plotting scalars: Viewer0D

Scalars or `Data0D` data are the simplest form of data. However, just displaying numbers is somewhat lacking (in particular when one want to compare data evolution over time, or parameter change...). This is why it is important to keep track of the history of the scalar values. The `Viewer0D`, see below, has such an history as well as tools to keep track of the maximal reached value.

In [6]:
%gui qt
import numpy as np

from pymodaq.utils.plotting.data_viewers.viewer0D import Viewer0D
from pymodaq.utils.data import DataRaw

dwa = DataRaw('my_scalar', data=[np.array([10.6]), np.array([-4.6])])
viewer0D = Viewer0D()


In [7]:
viewer0D.show_data(dwa)

![Showing scalars](plotting_data/viewer0D_simple.png)

Well not much can be seen except for the numbers printed on the right (shown by clicking on the dedicated button ![](plotting_data/viewer0D_123.png)). But what if I call several time the `show_data` method to display evolving signal?

In [8]:
for ind in range(100):
    dwa = DataRaw('my_scalar', data=[np.sin([ind / 100 * 2*np.pi]),
                                     np.sin([ind / 100 * 2*np.pi + np.pi/4])],
                 labels=['mysinus', 'my_dephased_sinus'])
    viewer0D.show_data(dwa)

![Showing an history of scalars, together with their min and max values (dashed lines)](plotting_data/viewer0D_sinus.png)

You immediately see the usefulness of such an history, allowing for instance to optimize signals when tweaking a parameter especially if you use the dashed lines, triggered by ![Min/Max dashed lines](plotting_data/viewer0D_min_max.png), showing the values of the min and max reached values.


## Plotting vectors/waveforms: Viewer1D

When increasing complexity, one get one dimensional data. It has one more important metadata, its axis. Properly defining the data object will translate into rich plots:

In [9]:
from pymodaq.utils import math_utils as mutils
from pymodaq.utils.data import Axis

axis = Axis('my axis', units='my units', data=np.linspace(-10000, 10000, 100))

dwa = DataRaw('my_1D_data', data=[mutils.gauss1D(axis.get_data(), 3300, 2500),
                                  mutils.gauss1D(axis.get_data(), -4000, 1500) * 0.5],
              labels=['a gaussian', 'another gaussian'],
              axes=[axis])
dwa.plot('qt')

..note::
  One can directly call the method `plot` on a data object, PyMoDAQ will determine which data viewer to use.

![Showing Data1D](plotting_data/viewer1D.png)

You can see the legends correspond to the data labels, while the axis shows both the label and the units in scientific notation (notice the k before 'my units' standing for kilo).

As for the buttons in the toolbar (you can try them from the notebook):

* ![Showing Data1D](plotting_data/viewer1D_roi.png): opens the ROI (region of interest) manager, to load, save and define ROI to apply to the data. This will create croped Data1D and Data0D for the application of an operation on the croped data such as *mean*, *sum*, *std*... See figure below, showing the mean value on the bottom panel. ROI can be applied to one of the trace or to both as reflected by the legends
* ![Showing Data1D](plotting_data/viewer1D_crosshair.png): activate the croshair (yellow vertical line) that can be grabed and translated. The data at the crosshair position is printed on the right of the toolbar.
* ![Showing Data1D](plotting_data/viewer1D_zoom.png): fix the horizontal/vertical aspect ratio (usefull for xy plot see below)
* ![Showing Data1D](plotting_data/viewer1D_dot.png): as shown on the figure below, one can switch between solid line or only dots.
* ![Showing Data1D](plotting_data/viewer1D_xy.png): when data contains two waveforms, using this button will display them in XY mode.
* ![Showing Data1D](plotting_data/viewer1D_overlay.png): when activated, an overlay of the current data will be depicted with a dash line. 
* ![Showing Data1D](plotting_data/viewer1D_sort.png): if the axis data is not monotonous, data will be represented as a scrambled solid line, using this button will reorder the data by ascending values of its axis. See below and figure xx
* ![Showing Data1D](plotting_data/viewer1D_roi_select.png): extra ROI that can be used independantly of the ROI manager

![Showing Data1D](plotting_data/viewer1D_with_roi_crosshair_dot.png)

If the axis data is not monotonous, data will be represented as a scrambled solid line, for instance:

In [10]:
axis_shuffled_array = axis.get_data() 
np.random.shuffle(axis_shuffled_array)
axis_shuffled = Axis('my axis', units='my units', data=axis_shuffled_array)

dwa = DataRaw('my_1D_data', data=[mutils.gauss1D(axis_shuffled.get_data(), 3300, 2500),
                                  mutils.gauss1D(axis_shuffled.get_data(), -4000, 1500) * 0.5],
              labels=['a gaussian', 'another gaussian'],
              axes=[axis_shuffled])
dwa.plot('qt')

![Showing Data1D](plotting_data/viewer1D_shuffled.png)

The scrambled lines (left) still represents Gaussians, it is just that the random ordering scrambled the lines. If one remove the lines by clicking the `dot only` button, the Gaussians reappear (middle). They reappear also after pressing the sort button (right).

## Plotting 2D data

2D data can be either an image (pixels on a regular grid) or as a collection of scalars with XY coordinates. PyMoDAQ introduce therefore the notion of "uniform" data for the former and "spread" data for the later. They can however be transparently plotted on the same `Viewer2D` data viewer. One will first show both cases before discussing the `Viewer2D` toolbar.  

### Uniform data
Let's generate data displaying a 2D Gaussian distribution:

In [11]:
# generating uniform 2D data
NX = 100
NY = 50
x_axis = Axis('xaxis', 'xunits', data=np.linspace(-20, 20, NX), index=1)
y_axis = Axis('yaxis', 'yunits', data=np.linspace(20, 40, NY), index=0)

data_arrays_2D = [mutils.gauss2D(x_axis.get_data(), -5, 10, y_axis.get_data(), 25, 2),
                 mutils.gauss2D(x_axis.get_data(), 5, 5, y_axis.get_data(), 30, 8)]
data2D = DataRaw('data2DUniform', data=data_arrays_2D, axes=[x_axis, y_axis],
                labels=['red gaussian', 'green gaussian'])
data2D.plot('qt')

![Showing Data2D](plotting_data/viewer2D_uniform.png)

The bottom and left axes correspond to the image pixels while the left and top one correspond to the real physical axes defined in the data object. When several arrays are included into the data object, they will be displayed as RGB layers. Data visibility can be set using the red/green (blue) buttons. If only one array is used, the color will be white.

### Spread Data

Spread 2D data are typically what you get when doing a `Spread` or `Tabular` 2D scan, see :ref:`scanner_paragrah`. By the way, `Spread` or `Tabular` 1D scan would typically give the scrambled plot on figure :numref:`XXX`. Let's generate and plot such 2D data

In [14]:
# generating Npts of spread 2D data
N = 100
x_axis_array = np.random.randint(-20, 50, size=N)
y_axis_array = np.random.randint(20, 40, size=N)
x_axis = Axis('xaxis', 'xunits', data=x_axis_array, index=0, spread_order=0)
y_axis = Axis('yaxis', 'yunits', data=y_axis_array, index=0, spread_order=1)

data_list = []
for ind in range(N):
    data_list.append(mutils.gauss2D(x_axis.get_data()[ind], 10, 15,
                                    y_axis.get_data()[ind], 30, 5))
data_array = np.squeeze(np.array(data_list))

data2D_spread = DataRaw('data2DSpread', data=[data_array],
                         axes=[x_axis, y_axis],
                         distribution='spread',
                         nav_indexes=(0,))
data2D_spread.plot('qt')

In [16]:
print(data_array.shape)

(100,)


### Toolbar

As for the buttons in the toolbar (you can try them from the notebook):

* ![Viewer2D](plotting_data/viewer2D_rgb.png): 
* ![Viewer2D](plotting_data/viewer2D_autoscale.png):
* ![Viewer2D](plotting_data/viewer2D_histogram.png):
* ![Viewer2D](plotting_data/viewer2D_roi.png):
* ![Viewer2D](plotting_data/viewer2D_isocurve.png):
* ![Viewer2D](plotting_data/viewer2D_aspect.png):
* ![Viewer2D](plotting_data/viewer2D_crosshair.png):
* ![Viewer2D](plotting_data/viewer2D_roi_select.png):
* ![Viewer2D](plotting_data/viewer2D_orientation.png):
* ![Viewer2D](plotting_data/viewer2D_legend.png):